In [ ]:
from flask import Flask, render_template, request
from PIL import Image
import numpy as np
import pickle

app = Flask(__name__)


labels = ['rice leaf roller', 'rice leaf caterpillar', 'paddy stem maggot', 'asiatic rice borer', 'yellow rice borer',
                'rice gall midge', 'Rice Stemfly', 'brown plant hopper', 'white backed plant hopper', 'small brown plant hopper',
                'rice water weevil', 'rice leafhopper', 'grain spreader thrips', 'rice shell pest', 'grub',
                'mole cricket', 'wireworm', 'white margined moth', 'black cutworm', 'large cutworm',
                'yellow cutworm', 'red spider', 'corn borer', 'army worm', 'aphids',
                'Potosiabre vitarsis', 'peach borer', 'english grain aphid', 'green bug', 'bird cherry-oataphid',
                'wheat blossom midge', 'penthaleus major', 'longlegged spider mite', 'wheat phloeothrips', 'wheat sawfly',
                'cerodonta denticornis', 'beet fly', 'flea beetle', 'cabbage army worm', 'beet army worm',
                'Beet spot flies', 'meadow moth', 'beet weevil', 'sericaorient alismots chulsky', 'alfalfa weevil',
                'flax budworm', 'alfalfa plant bug', 'tarnished plant bug', 'Locustoidea',
                'lytta polita', 'legume blister beetle', 'blister beetle', 'therioaphis maculata Buckton', 'odontothrips loti',
                'Thrips', 'alfalfa seed chalcid', 'Pieris canidia', 'Apolygus lucorum', 'Limacodidae',
                'Viteus vitifoliae', 'Colomerus vitis', 'Brevipoalpus lewisi McGregor', 'oides decempunctata',
                'Polyphagotars onemus latus', 'Pseudococcus comstocki Kuwana', 'parathrene regalis', 'Ampelophaga',
                'Lycorma delicatula', 'Xylotrechus', 'Cicadella viridis', 'Miridae',
                'Trialeurodes vaporariorum', 'Erythroneura apicalis', 'Papilio xuthus', 'Panonchus citri McGregor',
                'Phyllocoptes oleiverus ashmead', 'Icerya purchasi Maskell', 'Unaspis yanonensis', 'Ceroplastes rubens',
                'Chrysomphalus aonidum', 'Parlatoria zizyphus Lucus', 'Nipaecoccus vastalor', 'Aleurocanthus spiniferus',
                'Tetradacus c Bactrocera minax', 'Dacus dorsalis(Hendel)', 'Bactrocera tsuneonis', 'Prodenia litura',
                'Adristyrannus', 'Phyllocnistis citrella Stainton', 'Toxoptera citricidus', 'Toxoptera aurantii',
                'Aphis citricola Vander Goot', 'Scirtothrips dorsalis Hood', 'Dasineura sp', 'Lawana imitata Melichar',
                'Salurnis marginella Guerr', 'Deporaus marginatus Pascoe', 'Chlumetia transversa',
                'Mango flat beak leafhopper', 'Rhytidodera bowrinii white', 'Sternochetus frigidus', 'Cicadellidae']



class_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']

# Load the trained model
with open('cropheaalth.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Load the trained model
with open('trained_model.pkl', 'rb') as file:
    loaded_model1 = pickle.load(file)
    

# Preprocess the image
def preprocess_image(img):
    img = img.resize((150, 150))
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Preprocess the image
def preprocess_image_crop(img):
    img = img.resize((128, 128))
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array



@app.route('/')
def index():
    return render_template('index.html')

# Prediction page
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        # Get the user image from the request
        user_image = request.files['user_image']
        img = Image.open(user_image)

        # Preprocess the image
        preprocessed_img = preprocess_image_crop(img)

        # Make predictions
        predictions = loaded_model.predict(preprocessed_img)
        predicted_probs = predictions[0]
        predicted_index = np.argmax(predicted_probs)
        predicted_label = class_labels[predicted_index]

        # Render the prediction result
        return render_template('result.html', predicted_label=predicted_label)
    else:
        # Handle the GET request, which can be used for other purposes or display a form
        return render_template('predict.html')
    



@app.route('/pest_predict', methods=['GET', 'POST'])
def pest_predict():
    if request.method == 'POST':
        # Get the user image from the request
        user_image = request.files['user_image']
        img = Image.open(user_image)

        # Preprocess the image
        preprocessed_img = preprocess_image(img)

        # Make predictions
        predictions = loaded_model1.predict(preprocessed_img)
        predicted_probs = predictions[0]
        predicted_index = np.argmax(predicted_probs)
        predicted_label = labels[predicted_index]

        # Render the prediction result
        return render_template('result.html', predicted_label=predicted_label)
    else:
        # Handle the GET request, which can be used for other purposes or display a form
        return render_template('predict.html')   
    

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/services')
def services():
    return render_template('services.html')

@app.route('/farmer')
def farmer():
    return render_template('farmer.html')

@app.route('/contact')
def contact():
    return render_template('contact.html')

@app.route('/signin')
def signin():
    return render_template('signin.html')

@app.route('/connect_buyer')
def connect_buyer():
    return render_template('connect_buyer.html')

@app.route('/diseases_identification')
def diseases_identification():
    return render_template('diseases_identification.html')

@app.route('/pesticide_prediction')
def pesticide_prediction():
    return render_template('pesticide_prediction.html')

@app.route('/demand_forecasting')
def demand_forecasting():
    return render_template('demand_forecasting.html')

@app.route('/new_technologies')
def new_technologies():
    return render_template('new_technologies.html')

# Add more routes here...

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jul/2024 16:18:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/vendor/glightbox/css/glightbox.min.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/vendor/remixicon/remixicon.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/vendor/bootstrap-icons/bootstrap-icons.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/vendor/swiper/swiper-bundle.min.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/vendor/aos/aos.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 16:18:06] "GET /static/vendor/purecounter/purecounter_vanilla.js HT

1/1 [==============================] - 1s 622ms/step


127.0.0.1 - - [20/Jul/2024 16:18:41] "POST /predict HTTP/1.1" 200 -
